# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
from keras.datasets import cifar10
import pandas as pd
import numpy as np
from keras.utils import np_utils

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [ True]])

In [8]:
x_train.shape

(50000, 32, 32, 3)

In [9]:
y_train.shape

(50000, 1)

In [21]:
type(y_train)

numpy.ndarray

In [0]:
y_train_lt = (y_train < 5).flatten()
y_train_gt = (y_train >= 5).flatten()
y_test_lt = (y_test < 5).flatten()
y_test_gt = (y_test >= 5).flatten()

In [0]:
x_train_lt5 = x_train[y_train_lt]
x_train_gt5 = x_train[y_train_gt]
x_test_lt5 = x_test[y_test_lt]
x_test_gt5 = x_test[y_test_gt]


In [0]:
y_train_lt5 = y_train[y_train < 5]
y_train_gt5 = y_train[y_train >= 5] - 5
y_test_lt5 = y_test[y_test < 5]
y_test_gt5 = y_test[y_test >= 5] - 5

In [33]:
x_train_lt5.shape

(25000, 32, 32, 3)

In [35]:
x_test_lt5.shape

(5000, 32, 32, 3)

In [0]:
x_train_lt5 = (x_train_lt5.astype('float32'))/255
x_test_lt5 = (x_test_lt5.astype('float32'))/255
x_train_gt5 = (x_train_gt5.astype('float32'))/255
x_test_gt5 = (x_test_gt5.astype('float32'))/255

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_test_lt5 = np_utils.to_categorical(y_test_lt5, 5, dtype='float32')
y_train_lt5 = np_utils.to_categorical(y_train_lt5, 5, dtype='float32')
y_test_gt5 = np_utils.to_categorical(y_test_gt5, 5, dtype='float32')
y_train_gt5 = np_utils.to_categorical(y_train_gt5, 5, dtype='float32')

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Conv2D,MaxPooling2D

In [41]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=x_train.shape[1:], activation='relu'))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [43]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [51]:
model.fit(x_train_lt5, y_train_lt5, batch_size=32, epochs=50, validation_data=(x_test_lt5, y_test_lt5))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.1642 - acc: 0.9418 - val_loss: 0.5344 - val_acc: 0.8540
Epoch 2/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.1634 - acc: 0.9407 - val_loss: 0.5583 - val_acc: 0.8474
Epoch 3/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.1622 - acc: 0.9432 - val_loss: 0.5654 - val_acc: 0.8516
Epoch 4/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.1620 - acc: 0.9426 - val_loss: 0.5430 - val_acc: 0.8494
Epoch 5/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.1621 - acc: 0.9435 - val_loss: 0.5955 - val_acc: 0.8470
Epoch 6/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.1600 - acc: 0.9445 - val_loss: 0.5634 - val_acc: 0.8536
Epoch 7/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.1578 - acc: 0.9473 - val_loss: 0.6134 - v

In [53]:
model.evaluate(x_test_lt5,y_test_lt5)

5000/5000 [==============================] - 5s 1ms/step


[0.6252407586097717, 0.837]

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)         

In [55]:
model.layers

In [59]:
for layer in model.layers[:9]:
    layer.trainable = False
    print(layer.trainable, layer)

False <keras.layers.convolutional.Conv2D object at 0x7fc5cb064c88>
False <keras.layers.convolutional.Conv2D object at 0x7fc5cb064e48>
False <keras.layers.pooling.MaxPooling2D object at 0x7fc5ca7e9eb8>
False <keras.layers.core.Dropout object at 0x7fc5ca7e92e8>
False <keras.layers.convolutional.Conv2D object at 0x7fc5ca7e9208>
False <keras.layers.convolutional.Conv2D object at 0x7fc5ca7e9860>
False <keras.layers.pooling.MaxPooling2D object at 0x7fc5ca765ac8>
False <keras.layers.core.Dropout object at 0x7fc5ca7b11d0>
False <keras.layers.core.Flatten object at 0x7fc5ca780ac8>


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [60]:
model.fit(x_train_gt5, y_train_gt5, validation_data=(x_test_gt5, y_test_gt5), epochs=10, batch_size=32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
   32/25000 [..............................] - ETA: 2:03 - loss: 12.2035 - acc: 0.0000e+00

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


25000/25000 [==============================] - 116s 5ms/step - loss: 1.2413 - acc: 0.5018 - val_loss: 0.7602 - val_acc: 0.7086
Epoch 2/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.7634 - acc: 0.7151 - val_loss: 0.5426 - val_acc: 0.7972
Epoch 3/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.5980 - acc: 0.7841 - val_loss: 0.4685 - val_acc: 0.8256
Epoch 4/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.5153 - acc: 0.8126 - val_loss: 0.4797 - val_acc: 0.8252
Epoch 5/10
25000/25000 [==============================] - 116s 5ms/step - loss: 0.4611 - acc: 0.8324 - val_loss: 0.3820 - val_acc: 0.8634
Epoch 6/10
25000/25000 [==============================] - 116s 5ms/step - loss: 0.4237 - acc: 0.8462 - val_loss: 0.3766 - val_acc: 0.8660
Epoch 7/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.3847 - acc: 0.8602 - val_loss: 0.3571 - val_acc: 0.8702
Epoch 8/10
25000/25000 [=====================

In [61]:
model.evaluate(x_test_gt5, y_test_gt5)

5000/5000 [==============================] - 5s 1ms/step


[0.32349377188682554, 0.8874]

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [64]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [66]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [67]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [68]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vect = TfidfVectorizer(ngram_range=(1,3), min_df=2, max_df=0.5,stop_words='english')

In [0]:
x_train_dt = vect.fit_transform(twenty_train.data)
x_test_dt = vect.transform(twenty_test.data)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()


In [75]:
logreg.fit(x_train_dt, twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
ytrainpred = logreg.predict(x_train_dt)

In [0]:
ypred = logreg.predict(x_test_dt)

In [0]:
from sklearn.metrics import accuracy_score

In [79]:
accuracy_score(ypred, twenty_test.target)

0.9014647137150466

In [81]:
accuracy_score(ytrainpred, twenty_train.target)

0.9937970757642889